In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

## 处理缺失数据

在许多数据分析工作中，缺失数据是经常发生的。pandas的目标之一就是尽量轻松地处理缺失数据。
例如，pandas对象的所有描述性统计默认都不包括缺失数据。

缺失数据在pandas中呈现的方式有些不完美，但对于大多数用户可以保证功能正常。
对于数值数据，pandas使用浮点值NaN（Not a Number）表示缺失数据。我们称其为哨兵值，可以方便的检测出来：

In [3]:
string_data = Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

Python内置的None值在对象数组中也可以作为NA

In [5]:
string_data[0] = None
string_data.isna()
string_data.notna()

0    False
1     True
2    False
3     True
dtype: bool

pandas项目中还在不断优化内部细节以更好处理缺失数据，像用户API功能，例如pandas.isnull，去除了许多恼人的细节。

|方法|说明|
|----|---|
|`dropna`|根据各标签的值中是否存在缺失数据对轴标签进行过滤，可通过阈值调节对缺失值的容忍度|
|`fillna`|用指定值或插值方法（如 `ffill` 或 `bfill`）填充缺失数据|
|`isna`|返回一个含有布尔值的对象，这些布尔值表示哪些值是缺失值，该对象的类型与源类型一样|
|`notna`|`isna` 的否定式|

### 滤除缺失数据

过滤掉缺失数据的办法有很多种。你可以通过 `pandas.isna` 或布尔索引的手工方法，但 `dropna` 可能会更实用一些。
对于一个 `Series` ，`dropna` 返回一个仅含非空数据和索引值的`Series`：

In [6]:
data = Series([1, np.nan, 2, np.nan, 3])
data.dropna()

0    1.0
2    2.0
4    3.0
dtype: float64

这等价于：

In [7]:
data[data.notna()]

0    1.0
2    2.0
4    3.0
dtype: float64

而对于DataFrame对象，事情就有点复杂了。你可能希望丢弃全NA或含有NA的行或列。`dropna` 默认丢弃任何含有缺失值的行：

In [8]:
data = DataFrame([
        [1., 6.5, 3.], 
        [1., np.nan, np.nan], 
        [np.nan, np.nan, np.nan], 
        [np.nan, 6.5, 3.]
    ], columns=list('ABC'))
data

,A,B,C
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [9]:
cleaned = data.dropna()
cleaned

,A,B,C
0,1.0,6.5,3.0


传入how='all'将只丢弃全为NA的那些行

In [10]:
data.dropna(how='all')

,A,B,C
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


用这种方式丢弃列，只需传入`axis=1`即可：

In [11]:
data['D'] = np.nan
data

,A,B,C,D
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [12]:
data.dropna(axis=1, how='all')

,A,B,C
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


假设你只想留下一部分观测数据，可以用 `thresh` 参数实现此目的，`thresh` 参数的值表示剩下行或列：

In [13]:
data.dropna(thresh=2)

,A,B,C,D
0,1.0,6.5,3.0,NaN
3,NaN,6.5,3.0,NaN


### 填充缺失数据

你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数据），而是希望通过其他方式填补那些“空洞”。
对于大多数情况而言，`fillna` 方法是最主要的函数。通过一个常数调用 `fillna` 就会将缺失值替换为那个常数值：

In [14]:
df = DataFrame([
        [1., 6.5, 3.], 
        [1., np.nan, np.nan], 
        [np.nan, np.nan, np.nan], 
        [np.nan, 7.5, 8.]
    ], columns=list('ABC'))
df.fillna(0)

,A,B,C
0,1.0,6.5,3.0
1,1.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,7.5,8.0


若是通过一个字典调用 `fillna`，就可以实现对不同的列填充不同的值：

In [15]:
df.fillna({'A': 100})

,A,B,C
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,100.0,NaN,NaN
3,100.0,7.5,8.0


`fillna` 默认会返回新对象，但也可以对现有对象进行就地修改：

In [16]:
df.fillna({'A': 100}, inplace=True)
df

,A,B,C
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,100.0,NaN,NaN
3,100.0,7.5,8.0


对reindexing有效的那些插值方法也可用于`fillna`：

`method` 有两个值：`ffill` 和 `bfill` 。`ffill` 是根据NaN之前的值填充，`bfill` 是根据NaN之后的值填充。
同时可以通过 `limit` 参数设置填充的位数。 

In [17]:
df.fillna(method='ffill', limit=1)

,A,B,C
0,1.0,6.5,3.0
1,1.0,6.5,3.0
2,100.0,NaN,NaN
3,100.0,7.5,8.0


In [18]:
df.fillna(method='bfill')


,A,B,C
0,1.0,6.5,3.0
1,1.0,7.5,8.0
2,100.0,7.5,8.0
3,100.0,7.5,8.0
